In [2]:
import pandas as pd

## Загрузка и подготовка данных

In [3]:
# Путь к исходным данным
file_path = '/run/media/sergey/Archive_records/Disk_D/Data_analytics/ТЕСТОВЫЕ/01 585 Ювелирка Excel/Тестовое.csv'

In [4]:
# Чтение датайфрейма
df_shops = pd.read_csv(
    filepath_or_buffer=file_path,                                        # путь к файлу
    skiprows=1,                                                          # игнорирование строки с русскими названиями колонок
    encoding='utf-8',                                                    # кодировка 
    sep=',',                                                             # разделитель столбцов
    names=['period', 'shops_code', 'town', 'incoming_people', 'buyers'], # новые имена колонок
    converters={'incoming_people': lambda x: int(x.replace(',', '')),    # исключение разделительной запятой, и указание типа данных
                'buyers': lambda x: int(x.replace(',', ''))
               },
    dtype={'period': 'int',                                              # явное указание типов данных
           'shops_code': 'string', 
           'town': 'string'},
)

In [5]:
df_shops.dtypes

period                      int64
shops_code         string[python]
town               string[python]
incoming_people             int64
buyers                      int64
dtype: object

In [6]:
df_shops.head()	

,period,shops_code,town,incoming_people,buyers
0,1,1-01-145,Боровичи,3560,483
1,1,1-01-212,Великий Новгород,0,95
2,1,1-01-189,Великий Новгород,1896,237
3,1,1-01-228,Великий Новгород,3329,511
4,1,1-01-232,Великий Новгород,9837,975


## Фильтрация данных

### Вычисление магазинов только из Санкт-Петербурга.

In [7]:
df_shops.town.unique()

<StringArray>
[        'Боровичи', 'Великий Новгород',           'Чудово',
     'Великие Луки',            'Псков',          'Вологда',
            'Сокол',        'Череповец',   'Вышний Волочек',
             'Ржев',            'Тверь',        'Приозерск',
  'Санкт-Петербург',           'Выборг',        'Кронштадт',
     'Сосновый Бор',          'Гатчина',           'Волхов',
     'Петрозаводск',           'Тихвин',           'Кириши',
       'Никольское',            'Тосно',        'Кингисепп',
  'Новое Девяткино',           'Сланцы',       'Всеволожск',
          'Кировск',         'Отрадное',      'Калининград',
          'Коряжма',           'Котлас',       'Нарьян-Мар',
        'Сыктывкар',          'Воркута',           'Усинск',
             'Ухта',          'Апатиты',       'Кандалакша',
       'Мончегорск',         'Мурманск',      'Североморск',
      'Архангельск',           'Мирный',     'Северодвинск']
Length: 45, dtype: string

Видно, город Санкт-Петербург числится под одним названием "Санкт-Петербург". Исходя из этого и отфильтруем данные.

In [8]:
df_shops_SPb = df_shops[df_shops.town == 'Санкт-Петербург']
df_shops_SPb

,period,shops_code,town,incoming_people,buyers
23,1,1-01-107,Санкт-Петербург,5001,576
24,1,1-01-267,Санкт-Петербург,9072,773
25,1,1-01-269,Санкт-Петербург,2152,116
26,1,1-01-126,Санкт-Петербург,7264,939
27,1,1-01-250,Санкт-Петербург,7793,1061
...,...,...,...,...,...
216,2,1-01-223,Санкт-Петербург,2339,197
222,2,1-01-251,Санкт-Петербург,4672,478
223,2,1-01-162,Санкт-Петербург,2586,182
224,2,1-01-154,Санкт-Петербург,4144,331


## Поиск аномалий в данных.

Давайте взглянем на данные глазами.

In [9]:
df_shops_SPb.head(10)

,period,shops_code,town,incoming_people,buyers
23,1,1-01-107,Санкт-Петербург,5001,576
24,1,1-01-267,Санкт-Петербург,9072,773
25,1,1-01-269,Санкт-Петербург,2152,116
26,1,1-01-126,Санкт-Петербург,7264,939
27,1,1-01-250,Санкт-Петербург,7793,1061
28,1,1-01-236,Санкт-Петербург,2525,379
29,1,1-01-117,Санкт-Петербург,5512,721
30,1,1-01-211,Санкт-Петербург,0,23
31,1,1-01-203,Санкт-Петербург,0,36
32,1,1-01-171,Санкт-Петербург,3549,619


Из представленных данных видно что есть магазины, в которых количество посетителей равно 0, хотя покупатели есть. Возможно какая-то ошибка в исходных данных. Такое по просту не может быть чтобы были покупатели, а атсутствовали посетители. Покупатели уже являются посетителями.

---
Возможные причины:
- На этапе сбора данные не были зафиксированы.
- В процессе форматирования данных часть из них потерялась.
- Человеческий фактор.
---
Сейчас я принимаю решение исключить те магазины которые не соответствуют логике: количество посетителей равно 0, при присутствующих покупателях.

In [10]:
df_shops_SPb = df_shops_SPb[df_shops_SPb.incoming_people != 0]

### Определение магазинов, работающих в обоих периодах.

Для каждого магазина необходимо определить что он точно был в 1 и втором периоде. Для этого посчитаю количествоа. Надо как то определить что этот магазин внодил в оба периода и не входил только в один из двух. Для этого сглуппирую данные по shops_code.

In [11]:
df_shops_SPb

,period,shops_code,town,incoming_people,buyers
23,1,1-01-107,Санкт-Петербург,5001,576
24,1,1-01-267,Санкт-Петербург,9072,773
25,1,1-01-269,Санкт-Петербург,2152,116
26,1,1-01-126,Санкт-Петербург,7264,939
27,1,1-01-250,Санкт-Петербург,7793,1061
...,...,...,...,...,...
215,2,1-01-265,Санкт-Петербург,1787,348
216,2,1-01-223,Санкт-Петербург,2339,197
222,2,1-01-251,Санкт-Петербург,4672,478
223,2,1-01-162,Санкт-Петербург,2586,182


In [12]:
len(df_shops_SPb.shops_code.unique())

44

В данных мы имеем всего 54 уникальных магазина, работающих в СПб. Это нам пригодится для проверки.

Для фильтрации только тех магазинов, которые были как в первом и втором периоде проделаем следующее:
- сгруппируем данные по уникальному названию магазина;
- для каждого магазина сформируем списк уникальных периодов в которых он есть
- отфильтруем список магазинов, оставиа только те которые работали одновременно в обоих периодах

In [13]:
LFL_shops_SPb = df_shops_SPb.groupby('shops_code')['period'].unique().reset_index()
list_LFL_shops_SPb = list(LFL_shops_SPb[LFL_shops_SPb['period'].apply(lambda x: set(x) == {1, 2})].shops_code)

In [14]:
list_LFL_shops_SPb[:4]

['1-01-107', '1-01-117', '1-01-120', '1-01-122']

In [15]:
len(list_LFL_shops_SPb)

41

Исходя из полученных результатов, из 54 магазинов в Санкт-Петербурге в оба периода работало 51.

Сформируем итоговый датафрейм для дальнейших расчетов.

In [16]:
LFL_shops_SPb = df_shops_SPb[df_shops_SPb.shops_code.isin(list_LFL_shops_SPb)]
LFL_shops_SPb

,period,shops_code,town,incoming_people,buyers
23,1,1-01-107,Санкт-Петербург,5001,576
24,1,1-01-267,Санкт-Петербург,9072,773
25,1,1-01-269,Санкт-Петербург,2152,116
26,1,1-01-126,Санкт-Петербург,7264,939
27,1,1-01-250,Санкт-Петербург,7793,1061
...,...,...,...,...,...
215,2,1-01-265,Санкт-Петербург,1787,348
216,2,1-01-223,Санкт-Петербург,2339,197
222,2,1-01-251,Санкт-Петербург,4672,478
223,2,1-01-162,Санкт-Петербург,2586,182


### Проверка на повтояемость.

In [17]:
len(LFL_shops_SPb[LFL_shops_SPb.period == 1]['shops_code'].unique())

41

In [18]:
len(LFL_shops_SPb[LFL_shops_SPb.period == 2]['shops_code'].unique())

41

По результату получены необходимые данные: магазины только из города Санкт-Петербурга, работающие только в оба периода.

# Найти конверсию каждого магазина за оба периода.

Для этого необходимо:
- сгруппировать по периоду, магазину;
- посчитать конверию

Разделю DF на периоды.

In [19]:
LFL_shops_SPb_01 = LFL_shops_SPb[LFL_shops_SPb.period == 1][['shops_code', 'incoming_people', 'buyers']]
LFL_shops_SPb_02 = LFL_shops_SPb[LFL_shops_SPb.period == 2][['shops_code', 'incoming_people', 'buyers']]

In [20]:
LFL_shops_SPb_01.head()

,shops_code,incoming_people,buyers
23,1-01-107,5001,576
24,1-01-267,9072,773
25,1-01-269,2152,116
26,1-01-126,7264,939
27,1-01-250,7793,1061


In [21]:
gr_01 = LFL_shops_SPb_01.groupby(['shops_code']) \
.agg(
    incoming_people=('incoming_people', 'sum'),
    buyers=('buyers', 'sum'),
    conversion=('buyers', lambda x: (x.sum() / LFL_shops_SPb_01.loc[x.index, 'incoming_people'].sum() * 100)
               )
    ) \
.round(2) \
.reset_index()

In [40]:
gr_02 = LFL_shops_SPb_02.groupby(['shops_code']) \
.agg(
    incoming_people=('incoming_people', 'sum'),
    buyers=('buyers', 'sum'),
    conversion=('buyers', lambda x: (x.sum() / LFL_shops_SPb_02.loc[x.index, 'incoming_people'].sum() * 100)
               )
    ) \
.round(2) \
.reset_index()

In [29]:
gr_01.head()

,shops_code,incoming_people,buyers,conversion
0,1-01-107,5001,576,11.52
1,1-01-117,5512,721,13.08
2,1-01-120,2880,402,13.96
3,1-01-122,6450,740,11.47
4,1-01-126,7264,939,12.93


In [37]:
pd.merge(gr_01, gr_02, on='shops_code', how='inner')[['shops_code', 'conversion_x', 'conversion_y']]

,shops_code,conversion_x,conversion_y
0,1-01-107,11.52,10.32
1,1-01-117,13.08,12.25
2,1-01-120,13.96,10.06
3,1-01-122,11.47,9.74
4,1-01-126,12.93,11.25
5,1-01-127,7.96,7.12
6,1-01-134,15.08,11.94
7,1-01-137,14.14,13.53
8,1-01-146,10.57,8.15
9,1-01-153,12.00,12.00


In [47]:
conversion_gr_01 = (gr_01.conversion.sum() / gr_01.shops_code.count())
print(conversion_gr_01)

12.484390243902439


In [48]:
conversion_gr_02 = (gr_02.conversion.sum() / gr_02.shops_code.count())
print(conversion_gr_02)

10.597317073170732


In [60]:
difference_convertion = (conversion_gr_02 - conversion_gr_01).round(2)
print(f'''Разница между конверсией второго и первого периода составляет {difference_convertion}%. 
Что говорит о том что магазины стали более убыточны во втором периоде времени.''')

Разница между конверсией второго и первого периода составляет -1.89%. 
Что говорит о том что магазины стали более убыточны во втором периоде времени.
